In [1]:
import os

import numpy as np

from pathlib import Path
from collections import Counter

import runconfigs # type: ignore
import modelarchs # type: ignore

import nna.exp.megan as megan
from nna.exp import runutils


/scratch/enis/conda/envs/soundenv3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
# OVER WRITE CONFIG
runconfigs.TAXO_COUNT_LIMIT = 0

In [3]:


def prepare_dataset():

    taxo_count_limit = runconfigs.TAXO_COUNT_LIMIT
    sample_length_limit = runconfigs.SAMPLE_LENGTH_LIMIT
    taxonomy_file_path = runconfigs.TAXONOMY_FILE_PATH

    megan_labeled_files_info_path = runconfigs.MEGAN_LABELED_FILES_INFO_PATH

    csv4megan_excell_clenaed = runconfigs.CSV4MEGAN_EXCELL_CLEANED

    ignore_files = runconfigs.IGNORE_FILES

    excerpt_length = runconfigs.EXCERPT_LENGTH
    excell_names2code = runconfigs.EXCELL_NAMES2CODE
    dataset_name_v = runconfigs.DATASET_NAME_V

    audio_dataset, deleted_files = megan.preparedataset.run( # type: ignore
        megan_labeled_files_info_path,
        taxonomy_file_path,
        csv4megan_excell_clenaed,
        ignore_files,
        excerpt_length,
        sample_length_limit,
        taxo_count_limit,
        excell_names2code=excell_names2code,
        dataset_name_v=dataset_name_v)

    audio_dataset.load_audio_files(runconfigs.AUDIO_DATA_CACHE_PATH)
    audio_dataset.pick_channel_by_clipping()

    return audio_dataset, deleted_files

In [4]:
audio_dataset, _ = prepare_dataset()

/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite/meganLabeledFiles_wlenV1.txt
/scratch/enis/data/nna/labeling/megan/AudioSamplesPerSite
4 files are missing corresponding to excell entries
'-> 5 number of samples are DELETED due to ignore_files and missing_audio_files'
-> 415 samples DELETED because they are not in the excell

-> 0 samples DELETED because they do not have the taxo info coming from excell

-> classes that do not have enough data:
[REMOVED!]

-> classes that have enough data:
['other-biophony'] 56.0
['other-insect'] 140.0
['other-bird'] 661.0
['songbirds'] 392.0
['duck-goose-swan'] 183.0
['grouse-ptarmigan'] 59.0
['other-anthrophony'] 66.0
['other-mammal'] 0.0
['other-silence'] 20.0
['unknown-sound'] 2.0
['other-aircraft'] 107.0
['seabirds'] 1.0
['canids'] 1.0
['loons'] 29.0
['other-car'] 37.0
['other-flare'] 11.0
['other-rain'] 20.0
('-> 0 number of samples are deleted because their taxonomy category does not '
 'have enough data.')
-> classes that do not have e

In [5]:
from run import dataset_generate_samples
from run import data_to_samples


In [6]:
audio_dataset = dataset_generate_samples(audio_dataset,
                                             runconfigs.EXCERPT_LENGTH)

In [7]:
def taxo_counts(audio_dataset,prnt=False):
    taxo_dist=Counter()
    for sound_ins in audio_dataset.values():
        taxo_dist+=Counter({sound_ins.taxo_code:len(sound_ins.samples)})
    
    counted_items=list(taxo_dist.items())
    counted_items=sorted(counted_items)
    counted_items = [(i[0].split('.'),i[1]) for i in counted_items]
    if prnt:
        for i in counted_items:
            print(audio_dataset.taxonomy[i[0]][0],i[1])

    return counted_items

def print_counts_highest_taxo(audio_dataset):
    counted_items=taxo_counts(audio_dataset)
    
    first_level = Counter()
    for i in counted_items:
        first_level+={i[0][0]:i[1]}
    for m in first_level.items():
        if m[0]=='X':
            print('unknown sound',m[1])
        else:
            name_taxo = audio_dataset.taxonomy[[m[0],'0','0']][0]
            name_taxo=name_taxo.replace('other-','')
            print(name_taxo,m[1])


In [64]:
counted_items=taxo_counts(audio_dataset)

In [65]:
print_counts_highest_taxo(audio_dataset)

anthrophony 284
biophony 2691
geology 31
silence 53
unknown sound 24


In [106]:
taxo_dist = Counter()
location_ids = set([ i.location_id for i in audio_dataset.values()])
taxo_codes = set([ i.taxo_code for i in audio_dataset.values()])

location_ids = sorted(list(location_ids))
taxo_codes = sorted(list(taxo_codes))

table_dict = {loc:{taxo:0 for taxo in taxo_codes} for loc in location_ids}
total={taxo:0 for taxo in taxo_codes}
for sound_ins in audio_dataset.values():
    table_dict[sound_ins.location_id ][sound_ins.taxo_code]+=len(sound_ins.samples)
    total[sound_ins.taxo_code]+=len(sound_ins.samples)

In [91]:
from tabulate import tabulate



In [115]:
# create  table/matrix of counts in each label at each location

table=[[] for loc in range(len(location_ids)+1)]


for i,loc in enumerate(location_ids):
    table[i+1].append(loc)
    for taxo in taxo_codes:
        table[i+1].append(table_dict[loc][taxo])
table=sorted(table)

table[0].append('total')
for taxo in taxo_codes:
    table[0].append(total[taxo])
    
taxo_codes_names =[audio_dataset.taxonomy[i][0].replace('other-','')[:6] for i in taxo_codes] 
print(tabulate(table, headers=['loc']+taxo_codes_names))

loc      anthro    car    aircra    flare    biopho    bird    songbi    loons    seabir    duck-g    grouse    mammal    canids    insect    rain    silenc    unknow
-----  --------  -----  --------  -------  --------  ------  --------  -------  --------  --------  --------  --------  --------  --------  ------  --------  --------
total       103     44       120       17        84    1210       674       52         3       284       128         1         2       253      31        53        24
11            0      0         0        0         6      95        12        0         0         2         6         0         0         4       0         0         0
12            3      0         0        0        15     124         0        0         0         0         0         1         0         0       0         0         0
13            0      0         0        0         0      24         0        0         0         0         0         0         0         0       0         2         